## 렐루(ReLU) 함수

- 최근 활발히 사용되는 렐루 함수는 입력(x)이 음수일 때는 0을 출력하고, 양수일 때는 x를 출력한다.

- 경사 하강법(gradient descent)에 영향을 주지 않아 학습 속도가 빠르고, 기울기 소멸 문제가 발생하지 않는 장점이 있다.

- 렐루 함수는 일반적으로 은닉층에서 사용되며, 하이퍼볼릭 탄젠트 함수 대비 학습 속도가 6배 빠르다.

- 문제는 음수 값을 입력받으면 항상 0을 출력하기 때문에 학습 능력이 감소한다.

### 경사 하강법에 영향을 주지 않아 학습 속도가 빠르다는 의미

#### 역잔파 (Back propagation)

![Image](https://github.com/user-attachments/assets/71eead66-292d-4f01-a654-3fd212220392)

$$ s_3 = w_1 z_1 + w_2 z_2 + b_3 \quad
z_3 = \begin{cases}
0 & \text{if } s_3 \le 0 \\
s_3 & \text{if } s_3 > 0
\end{cases} $$

$w_1$을 업데이트 한다고 가정하면 손실함수를 $w_1$로 미분한 값이 필요하다.

$$\frac{\partial C^{(n)}}{\partial w_1} 
= \frac{d C^{(n)}}{d s_3} \cdot \frac{\partial s_3}{\partial w_1} 
= \frac{d C^{(n)}}{d s_3} \cdot z_1$$

$$ \frac{d C^{(n)}}{d s_3} 
= \frac{d C^{(n)}}{d z_3} \cdot \frac{d z_3}{d s_3}$$

$\frac{d z_3}{d s_3}$은 렐루 함수의 기울기를 구한 값으로 입력값이 양수이면 1, 음수이면 0인 상수를 출력하므로 학습 속도가 빠르면서 음수 값을 입력받으면 항상 0을 출력하기 때문에 학습 능력이 감소한다.

### 기울기 소멸 문제가 발생하지 않는다는 의미

#### 역전파 (Back propagation)

![Image](https://github.com/user-attachments/assets/9f060fd8-ed8e-4b14-b564-2c454c7b4c41)

활성화 함수가 sigmoid 함수이고 $w_{11}$를 업데이트 한다고 가정하면

$$ s_1 = w_{11} x_1 + w_{21} x_2 + b_1 \quad
z_1 = 1 / (1 + e^{-s_1}) $$

$w_{11}$을 업데이트 한다고 가정하면 손실함수를 $w_{11}$로 미분한 값이 필요하다.

$$\frac{\partial C^{(n)}}{\partial w_{11}} 
= \frac{d C^{(n)}}{d s_1} \cdot \frac{\partial s_1}{\partial w_{11}} 
= \frac{d C^{(n)}}{d s_3} \cdot x_1$$

$$ \frac{d C^{(n)}}{d s_1} 
= (\frac{d C^{(n)}}{d s_3}) \cdot \frac{\partial s_3}{\partial z_1} \cdot \frac{d z_1}{d s_1}$$

이전 과정에서 계산했던 $\frac{d C^{(n)}}{d s_3}$ 다시 쓰이는 것을 볼 수 있다. $\frac{d C^{(n)}}{d s_3} = \frac{d C^{(n)}}{d z_3} \cdot \frac{d z_3}{d s_3}$ 이므로 sigmoid 함수 $z_3$의 도함수가 쓰이는 것을 볼 수 있다. 이번 과정에서도 $\frac{d z_1}{d s_1}$ 즉, sigmoid 함수 $z_1$의 도함수가 쓰이는 것을 볼 수 있다. sigmoid의 도함수는 항상 0과 0.25 사이의 값으로 작아 만약 은닉층이 깊어져 연쇄적으로 곱해질 경우 기울기가 점점 작아지며 사라지는 현상, 즉 기울기 소멸 문제가 발생한다.  

반면 ReLU의 경우 도함수의 출력 값이 0과 1이므로 기울기가 사라지는 현상, 즉 기울기 소멸 문제가 발생하지 않는다.

## 소프트맥스(Softmax) 함수

- 소프트맥스 함수는 입력 값을 0~1 사이에 출력되도록 정규화하여 출력 값들의 총합이 항상 1이 되도록 한다.

- 소프트맥스 함수는 보통 딥러닝에서 출력 노드의 활성화 함수로 많이 사용한다.

- 수식으로 표현하면 다음과 같다.
$$
y_k = \frac{exp(a_k)}{\sum_{i=1}^n exp(a_i)}
$$

- n은 출력층의 뉴런 개수, $y_k$는 그 중 k번째 출력을 의미한다.

- 즉, 이 수식처럼 소프트맥스 함수의 분자는 입력 신호 $a_k$의 지수 함수, 분모는 모든 입력 신호의 지수 함수 합으로 구성된다.

## 렐루 함수와 소프트맥스 함수를 파이토치에서 구현하는 코드

In [1]:
import torch

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden) # 은닉층
        self.relu = torch.nn.ReLu(inplace=True)
        self.out = torch.nn.Linear(n_hidden, n_output) # 출력층
        self.softmax = torch.nn.Softmax(dim=n_output)
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x) # 은닉층을 위한 렐루 활성화 함수
        x = self.out(x)
        x = self.softmax(x) # 출력층을 위한 소프트맥스 활성화 함수
        return x